In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *
import tensorflow as tf

C:\Users\rstancut\AppData\Local\Continuum\anaconda2\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
training_data_df = pd.read_csv("Exercise Files/07/sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [3]:
# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir='Exercise Files/07/logs',
#     histogram_freq=5,
    write_graph=True
)

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

Epoch 1/50
 - 0s - loss: 0.0227
Epoch 2/50
 - 0s - loss: 0.0052
Epoch 3/50
 - 0s - loss: 0.0017
Epoch 4/50
 - 0s - loss: 5.5748e-04
Epoch 5/50
 - 0s - loss: 2.5953e-04
Epoch 6/50
 - 0s - loss: 1.7975e-04
Epoch 7/50
 - 0s - loss: 1.4039e-04
Epoch 8/50
 - 0s - loss: 1.0314e-04
Epoch 9/50
 - 0s - loss: 9.1074e-05
Epoch 10/50
 - 0s - loss: 1.0132e-04
Epoch 11/50
 - 0s - loss: 8.4940e-05
Epoch 12/50
 - 0s - loss: 6.2468e-05
Epoch 13/50
 - 0s - loss: 5.3496e-05
Epoch 14/50
 - 0s - loss: 5.0403e-05
Epoch 15/50
 - 0s - loss: 5.9816e-05
Epoch 16/50
 - 0s - loss: 4.4133e-05
Epoch 17/50
 - 0s - loss: 3.5091e-05
Epoch 18/50
 - 0s - loss: 3.6853e-05
Epoch 19/50
 - 0s - loss: 3.4347e-05
Epoch 20/50
 - 0s - loss: 3.5045e-05
Epoch 21/50
 - 0s - loss: 3.4173e-05
Epoch 22/50
 - 0s - loss: 3.8343e-05
Epoch 23/50
 - 0s - loss: 3.8071e-05
Epoch 24/50
 - 0s - loss: 3.8656e-05
Epoch 25/50
 - 0s - loss: 3.2387e-05
Epoch 26/50
 - 0s - loss: 2.7059e-05
Epoch 27/50
 - 0s - loss: 2.4506e-05
Epoch 28/50
 - 0s - lo

In [4]:
# Load the separate test data set
test_data_df = pd.read_csv("Exercise Files/07/sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

The mean squared error (MSE) for the test data set is: 7.801730826031416e-05


In [5]:
model_builder = tf.saved_model.builder.SavedModelBuilder("Exercise Files/07/exported_model")

inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'Exercise Files/07/exported_model\\saved_model.pb'


b'Exercise Files/07/exported_model\\saved_model.pb'